### Collect data and save to csv

#### Goal:  To save all data in a file the I think will be easier to use.  I hope to structure the csv's like a SQL database

In [20]:
# Python imports
import requests
import os

# 3rd party imports
import pandas as pd

# Custom imports
import utils.api_utils as api

In [21]:
json = api.get_json_from_api('https://fantasy.premierleague.com/api/bootstrap-static/')

fixtures_json = api.get_json_from_api(
  "https://api-football-beta.p.rapidapi.com/fixtures", 
  headers = {
    'x-rapidapi-host': os.getenv('RAPID_API_HOST'),
    'x-rapidapi-key':  os.getenv('RAPID_API_KEY')
    },
  querystring= {"season": "2021", "league":"39"}
    )

In [22]:
data = {}

for fixture in fixtures_json['response']:
  data[fixture['fixture']['id']] = {
    'date': fixture['fixture']['date'],
    'timestamp': fixture['fixture']['timestamp'],
    'venue': fixture['fixture']['venue']['id'],
    'home_team': fixture['teams']['home']['id'],
    'home_team_name': fixture['teams']['home']['name'],
    'away_team': fixture['teams']['away']['id'],
    'away_team_name': fixture['teams']['away']['name'],
    'home_score': fixture['score']['fulltime']['home'],
    'away_score': fixture['score']['fulltime']['away'],
  }

rapid_api_fixture_df = pd.DataFrame(data)
rapid_api_fixture_df = rapid_api_fixture_df.T
rapid_api_fixture_df.rename(columns={'home_team': 'api_home_team_id', 'away_team': 'api_away_team_id'}, inplace=True)

In [23]:
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

### Save gameweek info.

In [24]:
events_df = pd.DataFrame(json['events'])

gameweek_df = events_df[['id', 'deadline_time', 'data_checked']].set_index('id')

gameweek_df.to_csv('./csv/gameweek.csv')

### Save transfer stats

In [25]:
transfer_stats_by_week_df = events_df[['id', 'most_selected', 'most_transferred_in', 'most_captained', 'most_vice_captained']].set_index('id')
transfer_stats_by_week_df.to_csv('./csv/transfer_stats_by_week.csv')

### Save fantasy player stats

In [26]:
fpl_player_stats_df = events_df[['id', 'average_entry_score', 'highest_score', 'highest_scoring_entry']].set_index('id')
fpl_player_stats_df.to_csv('./csv/fpl_player_stats.csv')

#### game_settings nor phases relevant for data insight.

### Save team information

In [27]:
teams_df = pd.DataFrame(json['teams'])

team_info_df = teams_df[['id', 'name', 'short_name']].set_index('id')
team_info_df['name'].replace({"Man Utd":"Manchester United", "Man City": "Manchester City", "Spurs":"Tottenham"}, inplace=True)

rapid_api_team_info_df = rapid_api_fixture_df[['api_home_team_id', 'home_team_name']].to_numpy()
api_dict = {}
for i in rapid_api_team_info_df:
  if i[1] not in api_dict:
    api_dict[i[1]] = i[0]

team_info_df['api_id'] = team_info_df.apply(lambda x: api_dict[x['name']], axis=1)
print(team_info_df.head())
team_info_df.to_csv('./csv/team_info.csv')

           name short_name  api_id
id                                
1       Arsenal        ARS      42
2   Aston Villa        AVL      66
3     Brentford        BRE      55
4      Brighton        BHA      51
5       Burnley        BUR      44


### Save team strength.

In [28]:
team_strength_df = teams_df[['id', 'strength', 
  'strength_overall_home', 'strength_attack_home', 'strength_defence_home', 
  'strength_overall_away', 'strength_attack_away', 'strength_defence_away']].set_index('id')

team_strength_df.to_csv('./csv/team_strength.csv')

In [29]:
team_info_df = teams_df[['id', 'name', 'short_name']].set_index('id')

team_info_df.to_csv('./csv/team_info.csv')

### total_players not relevant

In [30]:
elements_df = pd.DataFrame(json['elements'])

pl_player_info_df = elements_df[[
  'id', 'team',  # reference info
  'first_name', 'second_name', 'web_name',  # Name
  'element_type', 'squad_number', 'photo' # Additional
  ]].set_index('id')
pl_player_ingame_stats_df = elements_df[[
  'id', 
  'minutes', 'yellow_cards', 'red_cards', 'penalties_missed', 'penalties_saved', # General stats
  'goals_scored', 'assists', # Attacking stats
  'clean_sheets', 'goals_conceded', 'saves', 'own_goals' # Defensive stats
  ]].set_index('id')

pl_player_info_df.to_csv('csv/pl_player_info.csv')
pl_player_ingame_stats_df.to_csv('csv/pl_player_ingame_stats.csv')

### Can also extract totals from ```json['elements']```, e.g. total_points.  I think it would be better to arrange info by gameweek and calculate totals. 

In [31]:
element_stats_df = pd.DataFrame(json['element_stats'])

element_stats_df.set_index('name', inplace=True)

element_stats_df.to_csv('csv/element_stats.csv')

In [32]:
element_types_df = pd.DataFrame(json['element_types'])

element_types_df.set_index('id')

element_info_df = element_types_df[[
  'id', 'plural_name', 'plural_name_short', 'singular_name'
]].set_index('id')

element_info_df.to_csv('csv/element_info.csv')

### Save Fixtures

In [33]:
rapid_api_fixture_df.to_csv('csv/fixtures.csv')